In [3]:
#import requests
#import bs4
#from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import config
from tqdm import tqdm_notebook
import tqdm
#from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
#from selenium import webdriver
import time
import pickle
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import spotipy.util as util
from scipy.sparse import coo_matrix

## Cleaning Data

In [95]:
with open('17375_mixes.pickle', 'rb') as handle:
    set_list_with_titles = pickle.load(handle)

In [92]:
expanded_set_lists=[]
for val in set_list_with_titles:
    expanded_set_lists.append([tup for tup in val])
expanded_set_lists=[x for sublist in expanded_set_lists for x in sublist]

In [ ]:
set_lists_df=pd.DataFrame(expanded_set_lists)
set_lists_df.columns=['set_title','set_list']
set_lists_df.set_title.drop_duplicates(inplace=True)

In [ ]:
set_lists_expanded_df=set_lists_df.set_index('set_title').explode('set_list').reset_index()
set_lists_expanded_df.dropna(inplace=True)
#gets rid of timestamps
set_lists_expanded_df.set_list=set_lists_expanded_df.set_list.map(lambda x :re.sub('^(\[[0-9]*:*[0-9]*])','',x))
set_lists_expanded_df.set_list=set_lists_expanded_df.set_list.map(lambda x :re.sub('(\[[0-9]*:*[0-9]*])','',x))
#replaces songs that are questionmarks with np.NaNs
set_lists_expanded_df.set_list.replace('^[\[ ].?\?',np.NaN,regex=True,inplace=True)
set_lists_expanded_df.set_list.replace('^\?',np.NaN,regex=True,inplace=True)

In [ ]:
set_lists_expanded_df['year'] = set_lists_expanded_df.set_title.map(lambda x:x.split(' - ')[0])
set_lists_expanded_df.set_title.replace('([0-9]+[-][0-9]+[-]?[0-9]+)','',regex=True,inplace=True)
#splitting out the dj name
set_lists_expanded_df['set_title_split']=set_lists_expanded_df.set_title.map(lambda x: x.split(' - ')[1] if len(x.split(' - '))>1 else x.split('-')[1])
set_lists_expanded_df['set_title_split']=set_lists_expanded_df['set_title_split'].map(lambda x:x.split('@')[0])

In [ ]:
set_lists_expanded_df.head()

In [ ]:
set_lists_expanded_df=set_lists_expanded_df[set_lists_expanded_df.set_title_split != 'VA']

In [ ]:
set_lists_expanded_df.dropna(inplace=True)

In [ ]:
def split_song(x):
    try:
        y=x.split(' - ')
        return y[1]
    except:
        #loads of ways that the songs are recorded and how missing songs are recorded too
        if '...' in x:
            return np.NaN
        if x =='X':
            return np.NaN
        if '?' in x:
            return np.NaN
        if 'Intro' in x:
            return np.NaN
        if '•' in x:
            return x.split('•')[1]
        #no space
        if '-' in x:
            return x.split('-')[1]
        
        
        return x

In [ ]:
set_lists_expanded_df['artist']=set_lists_expanded_df.set_list.map(lambda x:x.split(' - ')[0])
set_lists_expanded_df['song']=set_lists_expanded_df.set_list.map(split_song)

In [ ]:
#more different unusable song titles
filter_names_list= ['ID','?','Unknown','Unreleased','Intro','Untitled [Unreleased]','Untitled','Promo','? [Unreleased]','Outro']
set_lists_expanded_df['song']=set_lists_expanded_df['song'].map(lambda x: np.NaN if x in filter_names_list else x)

In [ ]:
set_lists_expanded_df.artist=set_lists_expanded_df.artist.replace('N/A',np.NaN)

In [ ]:
set_lists_expanded_df.dropna(inplace=True)

In [ ]:
set_lists_expanded_df.tail()
#can see that its taken a bit off of the some of the songs but think that may actually work in my favour
#a lot of the time spotify wont have the exact same song title (usually shorter on spotify) as mixesdb 
#so shortening names may be preferable


In [ ]:
set_lists_expanded_df.to_csv('songs_with_set_title_cleaned.csv',index=False)

In [134]:
set_lists_expanded_df=pd.read_csv('songs_with_set_title_cleaned.csv')

## Obtaining More Data

In [4]:
with open('150k_mixes.pickle','rb') as handle:
    more_sets_with_titles =pickle.load(handle)

In [5]:
expanded_sets=[]
for val in more_sets_with_titles:
    expanded_sets.append([tup for tup in val])
expanded_sets=[x for sublist in expanded_sets for x in sublist]

In [6]:
more_sets_df=pd.DataFrame(expanded_sets)
more_sets_df.columns=['set_title','set_list']
more_sets_df.set_title.drop_duplicates(inplace=True)

In [7]:
more_sets_df=more_sets_df.set_index('set_title').explode('set_list').reset_index()
more_sets_df.dropna(inplace=True)
#gets rid of timestamps
more_sets_df.set_list=more_sets_df.set_list.map(lambda x :re.sub('^(\[[0-9]*:*[0-9]*])','',x))
more_sets_df.set_list=more_sets_df.set_list.map(lambda x :re.sub('(\[[0-9]*:*[0-9]*])','',x))
#replaces songs that are questionmarks with np.NaNs
more_sets_df.set_list.replace('^[\[ ].?\?',np.NaN,regex=True,inplace=True)
more_sets_df.set_list.replace('^\?',np.NaN,regex=True,inplace=True)

In [8]:
more_sets_df['year'] = more_sets_df.set_title.map(lambda x:x.split(' - ')[0] if len(x.split(' - '))>1 else x.split('-')[0])
more_sets_df.year=more_sets_df.year.map(lambda x:x.split('-')[0])
more_sets_df.set_title.replace('([0-9]+[-][0-9]+[-]?[0-9]+)','',regex=True,inplace=True)
more_sets_df['set_title_split']=more_sets_df.set_title.map(lambda x: x.split(' - ')[1] if len(x.split(' - '))>1 else x)
more_sets_df['set_title_split']=more_sets_df['set_title_split'].map(lambda x:x.split('@')[0])
#filter out one set that was labelled wrong and i didnt like the music
more_sets_df=more_sets_df[~more_sets_df.set_title_split.str.contains('([0-9][1-9]-)')]

/Users/danielchow/opt/anaconda3/envs/rec_env/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [9]:
#we need to account for b2b sets
#going to say that if it was a b2b set then both artists played the song as cannot say for sure who actually did
#also can assume both djs like the song because they are playing together
def b2b_splitter(x):
    if 'b2b' in x:
        return x.split('b2b')
    if 'B2B' in x:
        return x.split('B2B')
    if ',' in x:
        return x.split(',')
    else:
        return [x]

In [10]:
more_sets_df.set_title_split = more_sets_df.set_title_split.map(b2b_splitter)

In [11]:
more_sets_df=more_sets_df.set_index(['set_title','set_list','year']).explode('set_title_split').reset_index()

In [12]:
#removing dj sets that had only 1 or 2 songs 
threshold = 0.00000001

counts = more_sets_df['set_title_split'].value_counts(normalize=True)
more_sets_df = more_sets_df.loc[more_sets_df['set_title_split'].isin(counts[counts > threshold].index), :]

In [13]:
#get rid of various artists
more_sets_df=more_sets_df[more_sets_df.set_title_split != 'VA']

In [14]:
more_sets_df.shape

(2783953, 4)

In [15]:
more_sets_df.dropna(inplace=True)

In [16]:
more_sets_df.shape

(2632472, 4)

In [17]:
def split_song(x):
    try:
        y=x.split(' - ')
        return y[1]
    except:
        #loads of ways that the songs are recorded and how missing songs are recorded too
        if '...' in x:
            return np.NaN
        if x =='X':
            return np.NaN
        if '?' in x:
            return np.NaN
        if 'Intro' in x:
            return np.NaN
        if '•' in x:
            return x.split('•')[1]
        #no space
        if '-' in x:
            return x.split('-')[1]
        
        
        return x

In [18]:
def split_artist(x):
    try:
        y=x.split(' - ')
        return y[0]
    except:
        #loads of ways that the songs are recorded and how missing songs are recorded too
        if '...' in x:
            return np.NaN
        if x =='X':
            return np.NaN
        if '?' in x:
            return np.NaN
        if 'Intro' in x:
            return np.NaN
        if '•' in x:
            return x.split('•')[0]
        #no space
        if '-' in x:
            return x.split('-')[0]
        
        
        return x

In [19]:
more_sets_df['artist']=more_sets_df.set_list.map(split_artist)
more_sets_df['song']=more_sets_df.set_list.map(split_song)

In [20]:
filter_names_list= ['ID','?','Unknown','Unreleased','Intro','Untitled [Unreleased]','Untitled','Promo','? [Unreleased]','Outro']
more_sets_df=more_sets_df[~more_sets_df.song.isin(filter_names_list)]
more_sets_df.artist=more_sets_df.artist.replace('N/A',np.NaN)
more_sets_df.dropna(inplace=True)

In [22]:
spotify_data_for_more_sets=pd.read_csv('mixes_db_tracks_with_spotify_copy.csv')

In [23]:
spotify_data_for_more_sets.dropna(inplace=True)

In [24]:
more_sets_df.sample(10)

,set_title,set_list,year,set_title_split,artist,song
1961713,- Ian Pooley - May Promo Mix (OFF Recordings ...,Robert Owens - Bring Down The Walls (Leo Zero ...,2011,Ian Pooley,Robert Owens,Bring Down The Walls (Leo Zero Edit)
2049097,"- Paul Oakenfold, Kenneth Thomas - Planet Per...",Robert Vadney - Fallen Angel's Symphony (Paul ...,2011,Paul Oakenfold,Robert Vadney,Fallen Angel's Symphony (Paul Oakenfold Remix)...
1408895,- Oneman - Rinse FM,Mondie - Straight (NAPPER VOCAL),2012,Oneman,Mondie,Straight (NAPPER VOCAL)
2201873,"- Laidback Luke - Super You & Me Radio, Club FG",Green Velvet - Harmageddon (The Battle),2010,Laidback Luke,Green Velvet,Harmageddon (The Battle)
1820976,"- MistaJam, DJ Cameo - Saturday Night Soundcl...",The Others - Africa VIP [Dub Police],2011,DJ Cameo,The Others,Africa VIP [Dub Police]
466510,- Solvynt - Lucky Lotus Online Electronic Mus...,Penguin Conspiracy - Tripswitch [Electrode],2015,Solvynt,Penguin Conspiracy,Tripswitch [Electrode]
621981,- Dreadmaul - Boundless Beatz Podcast 09,Dom & Roland & Amon Tobin - Sylo [Dom & Roland],2015,Dreadmaul,Dom & Roland & Amon Tobin,Sylo [Dom & Roland]
1473936,"- Friction, Audio - Drum'N'Bass Show",Tantrum Desire Feat. I-Kay - What Is Your Desi...,2012,Friction,Tantrum Desire Feat. I-Kay,What Is Your Desire (Dance Floor Pressure Mix)...
1190218,- Deep Space House - Deep Space House Show 055,[1:10:20] Tvardovsky - Jailbrake [Stereo Parad...,2013,Deep Space House,[1:10:20] Tvardovsky,Jailbrake [Stereo Paradise]
1332471,"- DK, Hidden Orchestra, Teenage DJ - Solid Steel",Hidden Orchestra - Spoken [Tru Thoughts],2013,DK,Hidden Orchestra,Spoken [Tru Thoughts]


In [25]:
#only want songs with spotify ids
more_sets_df_with_spotify=pd.merge(more_sets_df,spotify_data_for_more_sets,how='outer',left_on=['artist','song'],right_on=['artist','songs'])

In [26]:
#filling those without spotify info with blank
more_sets_df_with_spotify.fillna('-',inplace=True)

In [27]:
more_sets_df_with_spotify.set_title_split=more_sets_df_with_spotify.set_title_split.str.lstrip().str.rstrip()

In [53]:
more_sets_df_with_spotify.columns =['set_title','song_name_mixesdb','year','DJ','song_artist','song_name','spotify_song_name','spotify_id','preview','songs']

In [54]:
more_sets_df_with_spotify.to_csv('more_data_clean_with_spotify.csv',index=False)

## Syncing up songs with spotify

In [ ]:
def refresh():
    '''when spotify token times out this refreshes it'''
    global token_info, sp

    if sp_oauth.is_token_expired(token_info):
        token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
        token = token_info['access_token']
        sp = spotipy.Spotify(auth=token)

In [ ]:
#setting up credentials for accessing spotify
redirect_uri='https://www.google.com/'
scopes ='user-follow-modify'
client_id= config.client_id
client_secret =config.client_secret
username = '1143043561'

sp_oauth = oauth2.SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scopes,username=username)
token_info = sp_oauth.get_cached_token() 

if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']
else:
    token = token_info['access_token']
sp = spotipy.Spotify(auth=token)

In [ ]:
artists =set_lists_expanded_df.song_artist.tolist()
tracks =set_lists_expanded_df.song_name.tolist()

In [ ]:
print(len(artists),len(tracks))



spotify_song_names=[]
spotify_ids=[]
spotify_previews=[]
not_found_count =0
for ind, tup in tqdm_notebook(enumerate(zip(artists,tracks))):
    
    
    if ind%1500==0:
        print(ind)

    try:
        track_id = sp.search(q='artist:' + tup[0] + ' track:' + tup[1], type='track')
        if not track_id:
            if '[' in tup[1]:
                #accounting for the weird entries of songs and artists where they will be the same
                song_short = tup[1].split('[')[0]
                track_id = sp.search(q='artist:' + tup[0] + ' track:' + song_short, type='track')
                
        if not track_id:
            #same as above
            if tup[0]==tup[1]:
                track_id =sp.search(q=tup[0],type='track')
    except:
        refresh()
        track_id = sp.search(q='artist:' + tup[0] + ' track:' + tup[1], type='track')

        
    try:
    #some tracks cant be found with the search
        if track_id['tracks']['items'][0]['id']:
            spotify_ids.append(track_id['tracks']['items'][0]['id'])
        else:
            spotify_ids.append(np.NaN)

        if track_id['tracks']['items'][0]['preview_url']:
            spotify_previews.append(track_id['tracks']['items'][0]['preview_url'])
        else:
            spotify_previews.append(np.NaN)

        if track_id['tracks']['items'][0]['name']:
            spotify_song_names.append(track_id['tracks']['items'][0]['name'])
        else:
            spotify_song_names.append(np.NaN)


    except:
        spotify_song_names.append(np.NaN)
        spotify_ids.append(np.NaN)
        spotify_previews.append(np.NaN)
        not_found_count +=1
print('num songs not found :',not_found_count)

In [ ]:
#create a df out of the songs with spotify info
limited_df =pd.DataFrame([spotify_song_names,spotify_ids,spotify_previews]).T

In [ ]:
limited_df.columns=['spotify_song_name','spotify_id','preview']
limited_df['artist']=artists
limited_df['songs']=tracks

In [ ]:
limited_df.dropna(subset=['spotify_id'],inplace=True)
limited_df.drop_duplicates(inplace=True)

In [ ]:
limited_df.to_csv('mixes_db_tracks_with_spotify.csv',index=False)

## Preparing dataset for reccomendations

In [55]:
set_lists_expanded_df=pd.read_csv('songs_with_set_title_cleaned.csv')
spotify_song_info= pd.read_csv('mixes_db_tracks_with_spotify.csv')

In [56]:
set_lists_expanded_df.columns=['set_title','song_name_mixesdb','DJ','song_artist','song_name']

In [57]:
#kept all values as lists so i can explode
#same process as above in splitting apart b2b sets
set_lists_expanded_df.DJ=set_lists_expanded_df.DJ.map(b2b_splitter)

In [58]:
cols=set_lists_expanded_df.columns.tolist()
cols.remove('DJ')
set_lists_expanded_df=set_lists_expanded_df.set_index(keys=cols).explode('DJ').reset_index()
set_lists_expanded_df['year']=set_lists_expanded_df.set_title.map(lambda x:x.split(' - ')[0] if len(x.split(' - '))>1 else x.split('-')[0])

In [67]:
sets_with_spotify_part_1= pd.merge(set_lists_expanded_df,spotify_song_info,how='right',left_on=['song_artist','song_name'],right_on=['artist','songs'])
#fill those without spotify info with blanks
sets_with_spotify_part_1.fillna('-',inplace=True)
#load in the second part of the spotify data made above
sets_with_spotify_part_2= pd.read_csv('more_data_clean_with_spotify.csv')
#make sure columns in same order
sets_with_spotify_part_1[sets_with_spotify_part_2.columns.tolist()]
full_sets_with_spotify=pd.concat([sets_with_spotify_part_1,sets_with_spotify_part_2])
full_sets_with_spotify.drop_duplicates(subset=['set_title','song_name_mixesdb','DJ'],inplace=True)
full_sets_with_spotify.drop(columns=['artist'],inplace=True)

In [71]:
#numbering the songs,users, and amount of plays so we can make a sparse matrix out of them
full_sets_with_spotify['user_nums'] = full_sets_with_spotify.groupby('DJ').ngroup().values
full_sets_with_spotify['song_nums'] = full_sets_with_spotify.groupby('song_name_mixesdb').ngroup().values
count_series = full_sets_with_spotify.groupby(['DJ', 'song_name_mixesdb']).size()
plays_df = count_series.to_frame(name = 'plays').reset_index()
full_sets_with_spotify=pd.merge(full_sets_with_spotify,plays_df,on=['DJ','song_name_mixesdb'],how='left')

In [72]:
full_sets_with_spotify.dropna(inplace=True)

In [73]:
full_sets_with_spotify.shape

(2713104, 13)

In [75]:
#we need to make sure the sparse matrix is not too sparse otherwise this will not work that well
#max sparsity around 98.5%

plays= full_sets_with_spotify['plays'].values
users=full_sets_with_spotify.user_nums.values
songs=full_sets_with_spotify.song_nums.values
Matrix_us=coo_matrix((plays, (songs, users))).tocsr()

# Number of possible DJ song interactions
matrix_size = Matrix_us.shape[0]*Matrix_us.shape[1] 
# Number of songs
num_songs = len(Matrix_us.nonzero()[0]) 
sparsity = 100*(1 - (num_songs/matrix_size))
print('matrix sparsity -',sparsity)

matrix sparsity - 98.98243301


In [76]:
full_sets_with_spotify.to_csv('mixesdb_df_for_recs.csv',index=False)